# 95/20 Ranking

Estimate top-coded incomes from PUMs data. It sounds like we should pull with a PUMA instead of MET2013, and translate the geography in the analysis.

In [182]:
import pandas

ct_census = pandas.read_csv("too_big/ss14hct.csv") ## Grab for every topcoded state

pums = pandas.read_csv("too_big/usa_00002.csv")
acs_14 = pandas.read_csv("hh_14.csv",encoding = "ISO-8859-1")

In [30]:
census_vars = ["PUMA","ADJINC","SERIALNO","HINCP","WGTP", "stpuma"]

#bridgeport_pumas = ["00100", "00101", "00102", "00103", "00104", "00105"]

ct_census["bridgeport"] = ct_census["PUMA"].apply(lambda x: ((x>=100) & (x<=105)))

bridgeport_2014 = ct_census[ct_census["bridgeport"]==True][census_vars]
bridgeport_2014["HHINC"] = pandas.to_numeric(bridgeport_2014["HINCP"],errors="coerce")
bridgeport_2014["ADJHH"] = bridgeport_2014.apply(lambda x: (x["ADJINC"]/1000000)*x["HHINC"], axis=1)

In [231]:
def multiply(row,income_col,weight_col,puma_col,tar):
    i = 0
    while i < row[weight_col]:
        new = {puma_col:row[puma_col],income_col:row[income_col]}
        tar.append(new)
        i+=1

def explode_weights(df,income_col, weight_col,puma_col):
    ## Make a list of HHINCOME for calculating
    ## a quantile where each household's HHINCOME
    ## is repeated in the series n times, where
    ## n == weight_col
    tar = []
    df.apply(lambda x: multiply(x,income_col,weight_col,puma_col,tar),axis=1)
    return tar

exploded_bridgeport = explode_weights(bridgeport_2014,"ADJHH","WGTP","PUMA")

In [32]:
pandas.DataFrame(exploded_bridgeport)["ADJHH"].quantile(.95, interpolation="lower")

558969.9774999999

# Hallelujah!

This is the number Brookings had! But from talking to the Census it sounds like they were wrong to adjust income to constant dollars, so we won't do that for our analysis:

In [33]:
exploded_bridgeport = explode_weights(bridgeport_2014,"HHINC","WGTP")
pandas.DataFrame(exploded_bridgeport)["HHINC"].quantile(.95)

554300.0

# What's left:

#1 Write the functions to streamline it
#2 Gather the necessary 2016 data: ACS & PUMS
#3 Put them where they need to be & fill out the 2016 data set
#4 Do the analysis for 2007, too, and get the over-time chart (or maybe figure out when EPI measured it's recession numbers from?)

* identify all the topcoded entries -- get the right state PUMs files ugh such a pain

* find the corresponding PUMAs in MABLE/GEOCORR, and figure out what to do if any of them cross a CBSA border

* extract the correct data from the PUMs files... a lot of this might be manual

In [34]:
acs_16 = pandas.read_csv("hh_16.csv", encoding = "ISO-8859-1")
mable = pandas.read_csv("geocorr15.csv", encoding = "ISO-8859-1")[1:]

top_coded = acs_16[acs_16["HD01_VD06"].isnull()]
bottom_coded = acs_16[acs_16["HD01_VD02"].isnull()]

top_coded["States"] = top_coded["GEO.display-label"].apply(lambda x: x.split(',')[1].split()[0].split('-'))
#top_coded["States"] = top_coded["States"].drop_duplicates()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [218]:
def digit_str(i,digits):
    s = str(i)
    fill = digits - len(s)
    j = 0
    while j < fill:
        s = '0'+s
        j += 1
    return s

def make_stpuma(st,puma):
    s_puma = digit_str(puma,5)
    return int(str(st)+s_puma)

make_stpuma(9,102)

900102

In [219]:
ct_census['stpuma'] = ct_census.apply(lambda x: make_stpuma(x["ST"],x["PUMA"]),axis=1)

In [220]:
mable['stpuma'] = mable.apply(lambda x: make_stpuma(x["state"],x["puma12"]),axis=1)

mable.head()

,state,puma12,cbsa,stab,cbsaname15,PUMAname,pop10,afact,stpuma
1,06,00101,41860.0,CA,"San Francisco-Oakland-Hayward, CA (Metro)",Alameda County (North)--Berkeley & Albany Cities,131119,1,600101
2,06,00102,41860.0,CA,"San Francisco-Oakland-Hayward, CA (Metro)",Alameda County (Northwest)--Oakland (Northwest...,158091,1,600102
3,06,00103,41860.0,CA,"San Francisco-Oakland-Hayward, CA (Metro)",Alameda County (Northeast)--Oakland (East) & P...,125229,1,600103
4,06,00104,41860.0,CA,"San Francisco-Oakland-Hayward, CA (Metro)",Alameda County (North Central)--Oakland City (...,124608,1,600104
5,06,00105,41860.0,CA,"San Francisco-Oakland-Hayward, CA (Metro)","Alameda County (West)--San Leandro, Alameda & ...",163534,1,600105


In [221]:
mable["cbsa"] = mable["cbsa"].apply(lambda x : pandas.to_numeric(x,errors="coerce"))
all_pumas = []


def get_pumas(geo_id,geo_corr):
    cbsa = geo_corr[geo_corr["cbsa"] == geo_id] 
    if len(cbsa) == 0:
        print(geo_id)
    target = {}
    for index,row in cbsa.iterrows(): # do i need "index"?
        target[int(row["stpuma"])] = float(row["afact"])
        all_pumas.append(int(row["stpuma"]))
    return target

top_coded["PUMAS"] = top_coded["GEO.id2"].apply(lambda x: get_pumas(x,mable))

640354


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [222]:
top_coded

,GEO.id,GEO.id2,GEO.display-label,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,States,PUMAS,95_Percentile
0,1600000US0640354,640354,"La Quinta city, California",27794.0,2167.0,55920,4923,90161,6006,146206,6942,NaN,NaN,[California],{},NaN
109,310M300US14460,14460,"Boston-Cambridge-Newton, MA-NH Metro Area",29225.0,345.0,60545,343,98251,581,154820,789,NaN,NaN,"[MA, NH]","{2500400: 0.346, 2500501: 1.0, 2500502: 1.0, 2...",240000.0
110,310M300US14500,14500,"Boulder, CO Metro Area",28368.0,911.0,55378,1196,91056,1521,148663,2650,NaN,NaN,[CO],"{800801: 0.211, 800802: 0.704, 800803: 1.0, 80...",219000.0
120,310M300US14860,14860,"Bridgeport-Stamford-Norwalk, CT Metro Area",32385.0,674.0,66551,945,110368,1393,191602,2122,NaN,NaN,[CT],"{900100: 1.0, 900101: 1.0, 900102: 1.0, 900103...",290000.0
259,310M300US20780,20780,"Edwards, CO Micro Area",40478.0,2582.0,66183,4343,94777,5378,139984,3936,NaN,NaN,[CO],{800400: 0.46},199300.0
417,310M300US27220,27220,"Jackson, WY-ID Micro Area",32402.0,5774.0,59526,4904,85468,5229,129621,9133,NaN,NaN,"[WY, ID]","{1601100: 0.078, 5600100: 0.196}",195000.0
506,310M300US31060,31060,"Los Alamos, NM Micro Area",47250.0,8691.0,88420,3310,123242,8281,177485,9948,NaN,NaN,[NM],{3500300: 0.143},191800.0
507,310M300US31080,31080,"Los Angeles-Long Beach-Anaheim, CA Metro Area",24426.0,149.0,48258,212,79542,294,130673,456,NaN,NaN,[CA],"{603701: 1.0, 603702: 1.0, 603703: 1.0, 603704...",250000.0
563,310M300US33260,33260,"Midland, TX Metro Area",31622.0,922.0,57037,2005,89892,3126,140136,4595,NaN,NaN,[TX],"{4802800: 0.047, 4803000: 1.0}",245000.0
601,310M300US34900,34900,"Napa, CA Metro Area",32283.0,1433.0,59536,1913,93005,2113,146169,3402,NaN,NaN,[CA],{605500: 1.0},236000.0


In [208]:
state_fips = mable.groupby("stab")[['state']].max()

state_fips.loc["CA"]["state"]

'06'

In [223]:
import zipfile,requests,io,os

def fetch_census(x):
    zip_path = "https://www2.census.gov/programs-surveys/acs/data/pums/2016/1-Year/csv_h"+x+".zip"
    r = requests.get(zip_path, stream=True)
    zip_ref = zipfile.ZipFile(io.BytesIO(r.content))
    dir_path = "too_big/2016/csv_h"+x+"/"
    zip_ref.extractall(dir_path)
    os.remove("too_big/2016/csv_h"+x+"/ACS2016_PUMS_README.pdf")

def download_unzip_census(ss_s):
    ss_s.apply(lambda x: fetch_census(x.lower()))
    
#fetch_census("ct")

In [224]:
def make_path(state):
    return ("too_big/2016/csv_h"+state+"/ss16h"+state+".csv")
    
def get_census_pums(ss_s,model):
    tar = pandas.DataFrame(columns=model.columns)
    for index, s in ss_s.iteritems():
        file = make_path(s.lower())
        new = pandas.read_csv(file) ## Grab for every topcoded state
        statefip = state_fips.loc[s]["state"]
        new["stpuma"] = new.apply(lambda x: make_stpuma(x["ST"],x["PUMA"]),axis=1)
        tar = pandas.concat([tar,new])
    return tar

In [47]:
import pickle

def write_obj(obj,filename):
    with open(filename, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_obj(filename):
    with open(filename, 'rb') as handle:
        b = pickle.load(handle)
        return b
    


In [225]:
s = pandas.Series(top_coded["States"].sum()).drop_duplicates().sort_values()
s = s[(s=="CT") | (s=="CO") | (s=="MA") | (s=="NH")]
if not (os.path.isfile("too_big/2016/csv_hco/ss16hco.csv")):
    download_unzip_census(s)

print("CENSUS")
census_pums = get_census_pums(s,ct_census)

print("CT")
ct_pums = get_census_pums(s[s=="CT"],ct_census)

CENSUS
CT


In [226]:
census_pums["bridgeport"] = census_pums["stpuma"].apply(lambda x: ((x>=900100) & (x<=900105)))
b = census_pums[census_pums["bridgeport"]]
print(len(b))
#b["HHINC"] = pandas.to_numeric(b["HINCP"],errors="coerce")
#exploded_bridgeport_2016 = explode_weights(b,"HINCP","WGTP")


4105


In [242]:
def pums_dataset(tc):
    if (os.path.isfile("too_big/exploded_census.pickle")):
        return load_obj("too_big/exploded_census.pickle")
    else:
        s = pandas.Series(tc["States"].sum()).drop_duplicates().sort_values()
        s = s[s!="California"]
        if not (os.path.isfile("too_big/2016/csv_hco/ss16hco.csv")):
            download_unzip_census(s)
        census_pums = get_census_pums(s,ct_census)
        print(len(census_pums))
        census_pums["relevant"] = census_pums["stpuma"].apply(lambda x: x in all_pumas)
        relevant_census = census_pums[census_pums["relevant"]]
        print(len(relevant_census))
        relevant_census["HHINC"] = pandas.to_numeric(relevant_census["HINCP"],errors="coerce")
        exploded_census = explode_weights(relevant_census,"HHINC","WGTP","stpuma")
        print("exploded_census", len(exploded_census))
        write_obj(exploded_census,"too_big/exploded_census.pickle")
        return exploded_census

exploded_census = pums_dataset(top_coded)
exploded_census = pandas.DataFrame(exploded_census)

770254
235083


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


exploded_census 21840632


In [245]:
# make dict of FIPS: pandas.DataFrame(pums rows)
# explode weights FIRST

def get_data(puma,percent,census):
    all_entries = census[census["stpuma"]==puma]
    if percent < 1: 
        s = all_entries.sample(frac=percent,random_state=0)
        return(s)
    return(all_entries)
    
def make_fips_data_dict(df,census):
    if (os.path.isfile("too_big/fips_dict.pickle")):
        return load_obj("too_big/fips_dict.pickle")
    else:
        d = {}
        for index, row in df.iterrows():
            tar = pandas.DataFrame(columns=census.columns)
            fip = row["GEO.id2"]
            pumas = row["PUMAS"]
            for p in pumas:
                new = get_data(p,pumas[p],census)
                tar = pandas.concat([tar,new])
            d[fip] = tar
        write_obj(d,"too_big/fips_dict.pickle")
        return d
    
fips_data_dict = make_fips_data_dict(top_coded,exploded_census)

In [246]:
fips_data_dict[14860]["HHINC"].quantile(.95)

482000.0

In [251]:
def get_quantile(fips,percent,d):
    data = d[fips]
    return(data["HHINC"].quantile(percent))

top_coded["95_Percentile"] = top_coded["GEO.id2"].apply(lambda x: get_quantile(x,.95,fips_data_dict))
top_coded["20_Percentile"] = top_coded["GEO.id2"].apply(lambda x: get_quantile(x,.2,fips_data_dict))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [252]:
top_coded

,GEO.id,GEO.id2,GEO.display-label,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,States,PUMAS,95_Percentile,20_Percentile
0,1600000US0640354,640354,"La Quinta city, California",27794.0,2167.0,55920,4923,90161,6006,146206,6942,NaN,NaN,[California],{},NaN,NaN
109,310M300US14460,14460,"Boston-Cambridge-Newton, MA-NH Metro Area",29225.0,345.0,60545,343,98251,581,154820,789,NaN,NaN,"[MA, NH]","{2500400: 0.346, 2500501: 1.0, 2500502: 1.0, 2...",298500.0,30300.0
110,310M300US14500,14500,"Boulder, CO Metro Area",28368.0,911.0,55378,1196,91056,1521,148663,2650,NaN,NaN,[CO],"{800801: 0.211, 800802: 0.704, 800803: 1.0, 80...",298000.0,30400.0
120,310M300US14860,14860,"Bridgeport-Stamford-Norwalk, CT Metro Area",32385.0,674.0,66551,945,110368,1393,191602,2122,NaN,NaN,[CT],"{900100: 1.0, 900101: 1.0, 900102: 1.0, 900103...",482000.0,34000.0
259,310M300US20780,20780,"Edwards, CO Micro Area",40478.0,2582.0,66183,4343,94777,5378,139984,3936,NaN,NaN,[CO],{800400: 0.46},235000.0,31700.0
417,310M300US27220,27220,"Jackson, WY-ID Micro Area",32402.0,5774.0,59526,4904,85468,5229,129621,9133,NaN,NaN,"[WY, ID]","{1601100: 0.078, 5600100: 0.196}",178080.0,23000.0
506,310M300US31060,31060,"Los Alamos, NM Micro Area",47250.0,8691.0,88420,3310,123242,8281,177485,9948,NaN,NaN,[NM],{3500300: 0.143},181500.0,11800.0
507,310M300US31080,31080,"Los Angeles-Long Beach-Anaheim, CA Metro Area",24426.0,149.0,48258,212,79542,294,130673,456,NaN,NaN,[CA],"{603701: 1.0, 603702: 1.0, 603703: 1.0, 603704...",269000.0,25000.0
563,310M300US33260,33260,"Midland, TX Metro Area",31622.0,922.0,57037,2005,89892,3126,140136,4595,NaN,NaN,[TX],"{4802800: 0.047, 4803000: 1.0}",245000.0,30000.0
601,310M300US34900,34900,"Napa, CA Metro Area",32283.0,1433.0,59536,1913,93005,2113,146169,3402,NaN,NaN,[CA],{605500: 1.0},269010.0,35000.0


In [282]:
%matplotlib inline
import math

def ninetyfive_twenty(row):
    if row["GEO.id2"] == 640354:
        return None
    if math.isnan(row["HD01_VD06"]):
        return (row["95_Percentile"]/row["20_Percentile"])
    else:
        return (row["HD01_VD06"]/row["HD01_VD02"])
    
inequality_chart = acs_16[["GEO.id2","GEO.display-label","HD01_VD02","HD01_VD06"]].join(top_coded[["GEO.id2","95_Percentile","20_Percentile"]].set_index("GEO.id2"),on="GEO.id2")    
inequality_chart[9520] = inequality_chart.apply(lambda x: ninetyfive_twenty(x),axis=1)
pop_16 = pandas.read_csv("ACS_16_1YR_B01003.csv",encoding = "ISO-8859-1")
inequality_chart = inequality_chart.join(pop_16[["GEO.id2","HD01_VD01"]].set_index("GEO.id2"),on="GEO.id2")
inequality_chart["us_metro"] = inequality_chart.apply(lambda x: ("Metro" in x["GEO.display-label"]) & (" PR " not in x["GEO.display-label"]) & (x["HD01_VD01"] > 564000),axis=1)

In [283]:
inequality_chart[inequality_chart["us_metro"]].sort_values(9520,ascending=False) ## join hte mabel population totals

,GEO.id2,GEO.display-label,HD01_VD02,HD01_VD06,95_Percentile,20_Percentile,9520,HD01_VD01,us_metro
120,14860,"Bridgeport-Stamford-Norwalk, CT Metro Area",32385.0,NaN,482000.0,34000.0,14.176471,944177.0,True
617,35620,"New York-Newark-Jersey City, NY-NJ-PA Metro Area",24732.0,NaN,302000.0,25200.0,11.984127,20153634.0,True
766,41860,"San Francisco-Oakland-Hayward, CA Metro Area",32235.0,NaN,394600.0,36000.0,10.961111,4679166.0,True
507,31080,"Los Angeles-Long Beach-Anaheim, CA Metro Area",24426.0,NaN,269000.0,25000.0,10.760000,13310447.0,True
611,35380,"New Orleans-Metairie, LA Metro Area",18379.0,194587.0,NaN,NaN,10.587464,1268883.0,True
768,41940,"San Jose-Sunnyvale-Santa Clara, CA Metro Area",39133.0,NaN,426900.0,41000.0,10.412195,1978816.0,True
559,33100,"Miami-Fort Lauderdale-West Palm Beach, FL Metr...",20387.0,209733.0,NaN,NaN,10.287585,6066387.0,True
546,32580,"McAllen-Edinburg-Mission, TX Metro Area",13966.0,143137.0,NaN,NaN,10.248962,849843.0,True
109,14460,"Boston-Cambridge-Newton, MA-NH Metro Area",29225.0,NaN,298500.0,30300.0,9.851485,4794447.0,True
414,27140,"Jackson, MS Metro Area",18354.0,179805.0,NaN,NaN,9.796502,580178.0,True


,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01
0,310M300US10140,10140,"Aberdeen, WA Micro Area",71628,NaN
1,310M300US10180,10180,"Abilene, TX Metro Area",170860,2842.0
2,310M300US10300,10300,"Adrian, MI Micro Area",98504,NaN
3,310M300US10420,10420,"Akron, OH Metro Area",702221,NaN
4,310M300US10460,10460,"Alamogordo, NM Micro Area",65410,NaN


# Qunitile Wages 2007-2016

Track either mean wages per quintile or quintile cut-offs over time, to show the stratification of the recession and the recovery. Probably want to do this for something like:

* CT (state)
* CT (urban areas)
* Bridgeport Metro Area
* New Haven Metro Area
* Bridgeport as city

OR do this or mean income urban vs non-urban w/in Connecticut?

So you need a call for each year -- and the url construction might change year to year, though I think it's mostly just 2016 that's special for some reason.

A bunch of maybe-useful api queries:

    * https://api.census.gov/data/2016/acs/acs1/variables.html
    * https://api.census.gov/data/2016/acs/acs1/examples.html
    * https://api.census.gov/data/2016/acs/acs1?get=B01001_001E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*
    * https://api.census.gov/data/2015/acs1/examples.html
    * https://api.census.gov/data/2016/
            

In [ ]:
import json,urllib,re,numpy,time,config

base = "https://api.census.gov/data/"

def construct_url(year,geo_type,geo_code):
    url = base+str(year)+"/acs/acs1?get="+config.fec_key+"&is_amended=False&per_page=100"
    return(url)

def lookup(cid, t=5):
    url = construct_url(cid)
    data = url_to_data(url)
    if t>0:
        time.sleep(t)
    if data is None:
        return None
    return data["results"]

## If the fn can't find a response at the url, it returns None
## That gets converted into an empty dataframe (not ideal), and
## and flagged for the second pass
def url_to_data(url):
    try:
        req = urllib.request.Request(url)
        req.add_header('User-agent', 'Mozilla 5.10')
        res = urllib.request.urlopen(req)
        data = json.load(res)
        return data
    except urllib.error.HTTPError as err:
        return None
    
lookup("C00406389")